## 1.ライブラリとデータの準備

#### 1.1 import

In [92]:
# ライブラリ

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt

In [93]:
# データ読み込み

train = pd.read_csv('train.csv')

In [94]:
# データ確認

train.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,tv
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌


#### 1.2 欠損値の確認
全てのカラムで欠損値がないことが確認できる

In [95]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1721 entries, 0 to 1720
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1721 non-null   int64 
 1   y        1721 non-null   int64 
 2   year     1721 non-null   int64 
 3   stage    1721 non-null   object
 4   match    1721 non-null   object
 5   gameday  1721 non-null   object
 6   time     1721 non-null   object
 7   home     1721 non-null   object
 8   away     1721 non-null   object
 9   stadium  1721 non-null   object
 10  tv       1721 non-null   object
dtypes: int64(3), object(8)
memory usage: 148.0+ KB


##### 感覚的な考察
【関係大きそう】
- stage
- match
- home, away, stadium

In [96]:
train["stage"] .value_counts()

stage
Ｊ２    1046
Ｊ１     675
Name: count, dtype: int64

In [97]:
train["match"].value_counts()

match
第６節第１日     54
第９節第１日     53
第１２節第１日    52
第３節第１日     51
第４節第１日     51
           ..
第３６節第４日     1
第３１節第２日     1
第３６節第３日     1
第３６節第２日     1
第３５節第２日     1
Name: count, Length: 72, dtype: int64

In [98]:
train["stadium"].value_counts()

stadium
味の素スタジアム                     64
ニンジニアスタジアム                   51
栃木県グリーンスタジアム                 50
松本平広域公園総合球技場                 49
ニッパツ三ツ沢球技場                   49
ＮＤソフトスタジアム山形                 49
鳴門・大塚スポーツパーク　ポカリスエットスタジアム    48
レベルファイブスタジアム                 48
フクダ電子アリーナ                    47
ケーズデンキスタジアム水戸                47
Ｓｈｏｎａｎ　ＢＭＷスタジアム平塚            46
シティライトスタジアム                  46
正田醤油スタジアム群馬                  45
うまかな・よかなスタジアム                45
富山県総合運動公園陸上競技場               45
岐阜メモリアルセンター長良川競技場            45
京都市西京極総合運動公園陸上競技場兼球技場        44
北九州市立本城陸上競技場                 43
大分銀行ドーム                      43
埼玉スタジアム２００２                  40
ヤマハスタジアム（磐田）                 40
万博記念競技場                      40
ノエビアスタジアム神戸                  39
等々力陸上競技場                     39
山梨中銀スタジアム                    39
ユアテックスタジアム仙台                 38
ＩＡＩスタジアム日本平                  37
日立柏サッカー場                     37
デンカビッグスワンスタジアム               37
エディオンスタジアム広島                 37
県立カシマサッカースタジアム               36


## 2.一旦、有効に思えるカラムだけを使って予測してみる
表記揺れは無いと仮定して、予測してみる

【手順】
- 必要カラムだけ取り出す（stage, match, stadium）
- カテゴリ変数のエンコーディング
- modelと学習・テストデータの用意
- train
- 評価

In [99]:
# 2.1 必要カラムだけ取り出す

train = train[["stage", "match", "stadium", "y"]]
train.head()

,stage,match,stadium,y
0,Ｊ１,第１節第１日,ユアテックスタジアム仙台,18250
1,Ｊ１,第１節第１日,豊田スタジアム,24316
2,Ｊ１,第１節第１日,万博記念競技場,17066
3,Ｊ１,第１節第１日,エディオンスタジアム広島,29603
4,Ｊ１,第１節第１日,札幌ドーム,25353


In [100]:
# 2.2 カテゴリ変数のエンコーディング

# stageカラムのエンコーディング
train["isJ1"] = train["stage"].str.replace("Ｊ１", "1").replace("Ｊ２", "0").astype(int)
train.drop(columns=["stage"], inplace=True)

# matchカラムのエンコーディング
train["section"] = train["match"].str.extract(r'第(\d+)節').astype(int) # 節の抽出
train["day"] = train["match"].str.extract(r'(\d+)日').astype(int) # 日の抽出

train.drop(columns=["match"], inplace=True)

# stadiumカラムのエンコーディング
train = pd.get_dummies(train, columns=["stadium"], drop_first=True)


In [101]:
train.head()

,y,isJ1,section,day,stadium_とりぎんバードスタジアム,stadium_エコパスタジアム,stadium_エディオンスタジアム広島,stadium_キンチョウスタジアム,stadium_ケーズデンキスタジアム水戸,stadium_シティライトスタジアム,...,stadium_豊田スタジアム,stadium_長崎県立総合運動公園陸上競技場,stadium_香川県立丸亀競技場,stadium_駒沢オリンピック公園総合運動場陸上競技場,stadium_鳴門・大塚スポーツパーク ポカリスエットスタジアム,stadium_鹿児島県立鴨池陸上競技場,stadium_ＩＡＩスタジアム日本平,stadium_ＮＡＣＫ５スタジアム大宮,stadium_ＮＤソフトスタジアム山形,stadium_Ｓｈｏｎａｎ ＢＭＷスタジアム平塚
0,18250,1,1,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,24316,1,1,1,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,17066,1,1,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,29603,1,1,1,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,25353,1,1,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [102]:
# 2.3 モデルと学習・テストデータの用意
import sklearn.model_selection

# 学習データの用意
y = train["y"]
X = train.drop(columns=["y"])
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y, test_size=0.2, random_state=42)

# モデルの用意
from sklearn import linear_model
model = linear_model.LinearRegression()

# 2.4 学習
model.fit(X_train, y_train)

# 2.5 評価
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.7767757524694141
0.76197084204222


## 3.テストデータを使って予測し、submitファイルを作成

- テストデータで予測
- submitファイルを作成

In [103]:
# 3.1 テストデータで予測

test = pd.read_csv('test.csv')
test = test[["stage", "match", "stadium"]]

# stageカラムのエンコーディング
test["isJ1"] = test["stage"].str.replace("Ｊ１", "1").replace("Ｊ２", "0").astype(int)
test.drop(columns=["stage"], inplace=True)

# matchカラムのエンコーディング
test["section"] = test["match"].str.extract(r'第(\d+)節').astype(int) # 節の抽出
test["day"] = test["match"].str.extract(r'(\d+)日').astype(int) # 日の抽出

test.drop(columns=["match"], inplace=True)

# stadiumカラムのエンコーディング
test = pd.get_dummies(test, columns=["stadium"], drop_first=True)

# trainとtestでカラムを揃える
missing_cols = set(X.columns) - set(test.columns)
for c in missing_cols:
    test[c] = 0 # 欠損カラムを0で埋める

test = test[X.columns]  # カラム順も揃える

# 予測
pred = model.predict(test)


In [104]:
print(pred[:5])  # 最初の5件を表示

[15361.71758395 16362.16829482 35991.00036889 11155.92203423
 25559.06971719]


In [105]:
# # 3.2 submitファイルの作成

# test = pd.read_csv('test.csv')
# submit = pd.DataFrame({
#     "id": test["id"],
#     "pred": pred    
# })

# submit.to_csv('submit_0727_1.csv', index=False, header=False)

# print(pred.shape)
# print(submit.shape)

## 4.他のモデルも試してみる
- LightGBM
- RandomForestRegressor
- CatBoost

In [106]:
# LightGBM
import lightgbm as lgb
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_model.fit(X_train, y_train)

print("-----------------------------------")

print("LightGBM train:", lgb_model.score(X_train, y_train))
print("LightGBM test :", lgb_model.score(X_test, y_test))

print("-----------------------------------")

# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
print("RF train:", rf_model.score(X_train, y_train))
print("RF test :", rf_model.score(X_test, y_test))

print("-----------------------------------")

# CatBoost
from catboost import CatBoostRegressor
cat_model = CatBoostRegressor(verbose=0, random_state=42)
cat_model.fit(X_train, y_train)
print("CatBoost train:", cat_model.score(X_train, y_train))
print("CatBoost test :", cat_model.score(X_test, y_test))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000154 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 123
[LightGBM] [Info] Number of data points in the train set: 1376, number of used features: 40
[LightGBM] [Info] Start training from score 10709.790698
-----------------------------------
LightGBM train: 0.8001939827445481
LightGBM test : 0.770366960035002
-----------------------------------
RF train: 0.9448926612845889
RF test : 0.8060978733900266
-----------------------------------
CatBoost train: 0.9040987785873408
CatBoost test : 0.8397623151080886


In [107]:
# 予測

lgb_pred = lgb_model.predict(test)
rf_pred = rf_model.predict(test)
cat_pred = cat_model.predict(test)

# submitファイルの作成

test = pd.read_csv('test.csv')
submit = pd.DataFrame({
    "id": test["id"],
    "pred": lgb_pred    
})

submit.to_csv('submit_0727_lgb.csv', index=False, header=False)

test = pd.read_csv('test.csv')
submit = pd.DataFrame({
    "id": test["id"],
    "pred": rf_pred    
})

submit.to_csv('submit_0727_rf.csv', index=False, header=False)

test = pd.read_csv('test.csv')
submit = pd.DataFrame({
    "id": test["id"],
    "pred": cat_pred    
})

submit.to_csv('submit_0727_cat.csv', index=False, header=False)
